## Import libraries

In [7]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

## Import data

In [6]:
data_train = pd.read_csv("data/train_dataset_hackathon_mkb.csv", sep = ';', encoding = 'cp1251')
data_test = pd.read_csv("data/test_dataset_hackathon_mkb.csv", sep = ';', encoding = 'cp1251')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17891 entries, 0 to 17890
Columns: 124 entries, id_contract to CITIZENSHIP_NAME
dtypes: float64(108), int64(4), object(12)
memory usage: 16.9+ MB
None


## EDA

In [8]:
profile = ProfileReport(data_train, title = "Train Data Stats")
profile.to_file("TrainDataReport.html")

In [9]:
profile = ProfileReport(data_test, title = "Test Data Stats")
profile.to_file("TestDataReport.html")